In [ ]:
import pandas as pd
data = pd.read_json('C:\study\研一上\作业\自然语言处理\data\data\_train.json',lines=True)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_json('C:\study\研一上\作业\自然语言处理\data\data\_train.json',lines=True,orient='records')
df = df[pd.notnull(df['label'])]

In [ ]:
print(df.head(11))

In [ ]:
with open('C:\study\研一上\作业\自然语言处理\data\data\label_list.txt', 'r') as f:
    lines = f.readlines()
    labels = []
    for line in lines:
        labels.append(line.strip())
    f.close()

In [ ]:
labels

In [ ]:
import re

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/{}\[\]\|@,;]')#括号直接去掉，而不是分成两个词
BAD_SYMBOLS_RE = re.compile('[^a-z #+_]')# 不显示数字了

In [ ]:
def clean_text(text):
    """
        text: a string

        return: modified initial string
    """
    #text = BeautifulSoup(text, "lxml").text  # HTML decoding
    text = text.lower()  # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text)  # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text)  # delete symbols which are in BAD_SYMBOLS_RE from text
    # text = ' '.join(word for word in text.split() if word not in STOPWORDS)  # delete stopwors from text
    return text

df['text'] = df['text'].apply(clean_text)

In [ ]:
df.iloc[0]['text']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd

In [ ]:
labels = df['label'].values
labels_dic = {}
for label_list in labels:
    for label in label_list:
        if label not in labels_dic:
            labels_dic[label] = 1
        else:
            labels_dic[label] += 1
labels_dic

In [ ]:
X_train = list(df['text'])
X_train

In [ ]:
y_train = df['label']

In [ ]:
y_train 

In [ ]:
y_train = list(y_train)

In [ ]:
vf = pd.read_json('C:\study\研一上\作业\自然语言处理\data\data2\_valid.json',lines=True)
vf = vf[pd.notnull(vf['label'])]

In [ ]:
print(vf.head(5))

In [ ]:
X_val = list(vf['text'])
X_val

In [ ]:
y_val = vf['label']
y_val

In [ ]:
y_val = list(y_val)

In [ ]:
CV = CountVectorizer(min_df=5, max_df=0.9, ngram_range=(1,2), token_pattern='(\S+)')
feature = CV.fit_transform(X_train)
print(feature)

In [ ]:
tfidf = TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1,2), token_pattern='(\S+)')
feature = tfidf.fit_transform(X_train)
print(feature)

In [ ]:
y_train

In [ ]:
mlb = MultiLabelBinarizer(classes=sorted(labels_dic.keys()))
y_train = mlb.fit_transform(y_train)
y_val = mlb.fit_transform(y_val)

In [ ]:
y_train

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

def evaluation(y_val, predicted):
    accuracy = accuracy_score(y_val, predicted)
    f1_score_macro = f1_score(y_val, predicted, average='macro')
    print("accuracy:", accuracy)
    print('f1_score_macro',f1_score_macro)

In [ ]:
pipeline1 = Pipeline([
    ('tfidf', TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1,2), token_pattern='(\S+)')),
    ('lsvc', OneVsRestClassifier(LinearSVC()))
])
pipeline1.fit(X_train,y_train)
predicted = pipeline1.predict(X_val)
evaluation(y_val, predicted)

In [ ]:
test_df = pd.read_csv('ceshitest.txt', sep='\0', header=None)
test_df.columns = ['text']

In [ ]:
test_df

In [ ]:
testSet = list(test_df['text'])

In [ ]:
test_predicted_labels = pipeline1.predict(testSet)

In [ ]:
test_predicted_labels

In [ ]:
test_original_labels = mlb.inverse_transform(test_predicted_labels)

In [ ]:
test_original_labels

In [ ]:
with open("test_labels.txt", "w") as file:
    for tol in test_original_labels:
        line = ','.join(tol)
        if not line.strip():
            line = "No_Mentioned"
        file.write(line + '\n')